In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
df = pd.read_json('movies/data/wikidata-movies.json.gz', orient='records', lines=True)
genres = pd.read_json('movies/data/genres.json.gz', orient='records', lines=True)

In [4]:
#remove movies without profit col
df = df[pd.notnull(df['made_profit'])]

In [5]:
#df['genre'] = df['genre'].str[0]
#genres.columns = ['genre_label', 'genre']

In [6]:
#merged = pd.merge(df, genres, on="genre")
#can't merge

In [7]:
df_stack = df['genre'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame()
df_stack.columns = ['wikidata_id']

In [8]:
df_genre = df_stack.reset_index().merge(genres, how="left", on="wikidata_id").set_index('index')


In [9]:
df_genre = pd.merge(df_genre, df, left_index=True, right_index=True).reset_index()


In [10]:
genres['profit'] = genres['genre_label'].apply(lambda x: df_genre[(df_genre['genre_label'] == x) & (df_genre['made_profit'] == True)].shape[0])
genres['noprofit'] = genres['genre_label'].apply(lambda x: df_genre[(df_genre['genre_label'] == x) & (df_genre['made_profit'] == False)].shape[0])

In [11]:
df_genre[(df_genre['country_of_origin'] == 'Q29') & (df_genre['made_profit'] == True)].shape[0]

7

In [12]:
df_genre = genres.sort_values(by='profit', ascending=False)
df_genre = df_genre[(df_genre['profit'] != 0) & (df_genre['noprofit'] != 0)]

In [13]:
# profit_percent = profit/total
df_genre['profit_percent'] = df_genre['profit'] / (df_genre['profit'] + df_genre['noprofit'])

In [14]:
df_genre

,genre_label,wikidata_id,profit,noprofit,profit_percent
2586,drama film,Q130232,226,46,0.830882
4664,action film,Q188473,198,41,0.828452
4632,comedy film,Q157443,162,27,0.857143
2782,science fiction film,Q471839,130,12,0.915493
112,film based on literature,Q52162262,119,16,0.881481
2601,fantasy film,Q157394,109,16,0.872000
1042,thriller film,Q2484376,108,14,0.885246
5248,adventure film,Q319221,104,15,0.873950
2653,horror film,Q200092,70,8,0.897436
4915,crime film,Q959790,60,11,0.845070


In [14]:
contingency = np.vstack([df_genre['profit'], df_genre['noprofit']])

In [15]:
chi2, p, dof, expected = stats.chi2_contingency(contingency)

In [16]:
p

0.008127575360670712

In [17]:
df

,based_on,cast_member,country_of_origin,director,enwiki_title,filming_location,genre,imdb_id,label,made_profit,main_subject,metacritic_id,original_language,publication_date,rotten_tomatoes_id,series,wikidata_id
6,NaN,"[Q5126010, Q3390414, Q5676024, Q237021]",Q29,[Q51892574],Orbiter 9,NaN,"[Q24925, Q21010853]",tt3469798,Orbiter 9,0.0,NaN,NaN,NaN,2017-04-07,m/orbiter_9,NaN,Q42577704
69,NaN,NaN,Q30,"[Q3384479, Q351884]",Despicable Me,NaN,[Q157443],tt1323594,Despicable Me,1.0,NaN,movie/despicable-me,Q1860,2010-06-20,m/1214097,NaN,Q4447
73,NaN,"[Q386349, Q1605965, Q3805579, Q271162, Q463226...",Q30,[Q2071],Eraserhead,[Q99],"[Q130232, Q200092, Q5967378]",tt0074486,Eraserhead,1.0,[Q906343],NaN,Q1860,1977-01-01,m/eraserhead,NaN,Q11618
81,Q17017426,"[Q117500, Q1376880, Q11930, Q311169, Q951634, ...",Q30,[Q11930],Dances with Wolves,[Q1558],"[Q130232, Q369747, Q21590660, Q21010853, Q319221]",tt0099348,Dances with Wolves,1.0,NaN,movie/dances-with-wolves,Q1860,1990-11-09,m/dances_with_wolves,NaN,Q20456
84,NaN,"[Q38111, Q211553, Q177311, Q8927, Q173399, Q20...",Q145,[Q25191],Inception,"[Q99, Q387047, Q17, Q90, Q1951, Q7275217, Q126...","[Q496523, Q471839, Q2484376, Q188473, Q319221]",tt1375666,Inception,1.0,"[Q544830, Q4425624]",movie/inception,Q1860,2010-07-08,m/inception,NaN,Q25188
92,NaN,"[Q229313, Q445772, Q727988, Q3163137, Q1372392...",Q16,[Q6385039],Mama (2013 film),"[Q172, Q133116, Q13939]",[Q200092],tt2023587,Mama,1.0,[Q80837],movie/mama,Q1860,2013-01-17,m/mama_2013,NaN,Q29446
113,Q243556,"[Q34012, Q41163, Q95043, Q464714, Q171736, Q32...",Q30,[Q56094],The Godfather,"[Q18438, Q60, Q1408, Q1460]","[Q130232, Q959790, Q7444356, Q21010853, Q52162...",tt0068646,The Godfather,1.0,[Q46952],movie/the-godfather,Q1860,1972-03-15,m/godfather,Q3225260,Q47703
125,NaN,"[Q483118, Q23547, Q108283, Q215072, Q270664, Q...",Q30,[Q483118],Argo (2012 film),"[Q406, Q65, Q43]","[Q622291, Q18620604]",tt1024648,Argo,1.0,"[Q589673, Q1032059, Q23036198]",movie/argo,Q1860,2012-01-01,m/argo_2012,NaN,Q59653
203,Q7857661,"[Q317343, Q57147, Q244674, Q343616, Q208649, Q...",Q145,[Q706475],12 Years a Slave (film),[Q34404],"[Q130232, Q645928, Q52162262]",tt2024544,12 Years a Slave,1.0,"[Q118382, Q8461]",movie/12-years-a-slave,Q1860,2013-08-30,m/12_years_a_slave,NaN,Q3023357
483,NaN,"[Q295803, Q200534, Q228865, Q200405, Q314133, ...",Q145,[Q191755],Only Lovers Left Alive,"[Q183, Q365]","[Q130232, Q1054574, Q2137852]",tt1714915,Only Lovers Left Alive,1.0,[Q46721],movie/only-lovers-left-alive,Q1860,2013-05-25,m/only_lovers_left_alive,NaN,Q3352751


In [18]:
df

,based_on,cast_member,country_of_origin,director,enwiki_title,filming_location,genre,imdb_id,label,made_profit,main_subject,metacritic_id,original_language,publication_date,rotten_tomatoes_id,series,wikidata_id
6,NaN,"[Q5126010, Q3390414, Q5676024, Q237021]",Q29,[Q51892574],Orbiter 9,NaN,"[Q24925, Q21010853]",tt3469798,Orbiter 9,0.0,NaN,NaN,NaN,2017-04-07,m/orbiter_9,NaN,Q42577704
69,NaN,NaN,Q30,"[Q3384479, Q351884]",Despicable Me,NaN,[Q157443],tt1323594,Despicable Me,1.0,NaN,movie/despicable-me,Q1860,2010-06-20,m/1214097,NaN,Q4447
73,NaN,"[Q386349, Q1605965, Q3805579, Q271162, Q463226...",Q30,[Q2071],Eraserhead,[Q99],"[Q130232, Q200092, Q5967378]",tt0074486,Eraserhead,1.0,[Q906343],NaN,Q1860,1977-01-01,m/eraserhead,NaN,Q11618
81,Q17017426,"[Q117500, Q1376880, Q11930, Q311169, Q951634, ...",Q30,[Q11930],Dances with Wolves,[Q1558],"[Q130232, Q369747, Q21590660, Q21010853, Q319221]",tt0099348,Dances with Wolves,1.0,NaN,movie/dances-with-wolves,Q1860,1990-11-09,m/dances_with_wolves,NaN,Q20456
84,NaN,"[Q38111, Q211553, Q177311, Q8927, Q173399, Q20...",Q145,[Q25191],Inception,"[Q99, Q387047, Q17, Q90, Q1951, Q7275217, Q126...","[Q496523, Q471839, Q2484376, Q188473, Q319221]",tt1375666,Inception,1.0,"[Q544830, Q4425624]",movie/inception,Q1860,2010-07-08,m/inception,NaN,Q25188
92,NaN,"[Q229313, Q445772, Q727988, Q3163137, Q1372392...",Q16,[Q6385039],Mama (2013 film),"[Q172, Q133116, Q13939]",[Q200092],tt2023587,Mama,1.0,[Q80837],movie/mama,Q1860,2013-01-17,m/mama_2013,NaN,Q29446
113,Q243556,"[Q34012, Q41163, Q95043, Q464714, Q171736, Q32...",Q30,[Q56094],The Godfather,"[Q18438, Q60, Q1408, Q1460]","[Q130232, Q959790, Q7444356, Q21010853, Q52162...",tt0068646,The Godfather,1.0,[Q46952],movie/the-godfather,Q1860,1972-03-15,m/godfather,Q3225260,Q47703
125,NaN,"[Q483118, Q23547, Q108283, Q215072, Q270664, Q...",Q30,[Q483118],Argo (2012 film),"[Q406, Q65, Q43]","[Q622291, Q18620604]",tt1024648,Argo,1.0,"[Q589673, Q1032059, Q23036198]",movie/argo,Q1860,2012-01-01,m/argo_2012,NaN,Q59653
203,Q7857661,"[Q317343, Q57147, Q244674, Q343616, Q208649, Q...",Q145,[Q706475],12 Years a Slave (film),[Q34404],"[Q130232, Q645928, Q52162262]",tt2024544,12 Years a Slave,1.0,"[Q118382, Q8461]",movie/12-years-a-slave,Q1860,2013-08-30,m/12_years_a_slave,NaN,Q3023357
483,NaN,"[Q295803, Q200534, Q228865, Q200405, Q314133, ...",Q145,[Q191755],Only Lovers Left Alive,"[Q183, Q365]","[Q130232, Q1054574, Q2137852]",tt1714915,Only Lovers Left Alive,1.0,[Q46721],movie/only-lovers-left-alive,Q1860,2013-05-25,m/only_lovers_left_alive,NaN,Q3352751
